# Drone Simulator

In [ ]:
%matplotlib notebook

from py_drone_module import Drone, DroneReference, DroneReferenceFunction, AdaptiveODEOptions
from py_drone_module import eul2quat, quatmultiply, quatconjugate
from DroneParamsAndMatricesBuilder import buildDroneParamsAndMatrices
from numpy import array, diag
from math import pi
import Reference
import DronePlotter
import matplotlib.pyplot as plt
import time

from ipywidgets import interact, interactive
import ipywidgets as widgets

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95vw !important; }</style>"))

### Attitude and Altitude Controller Settings

In [ ]:
Q_att = diag((139.6245112700232,
              139.6245112700232,
              15.2811761590895,
              1.1505204155597211,
              1.1505204155597211,
              0.1209919487616804,
              9.976475759487083e-08,
              9.976475759487083e-08,
              9.976475759487083e-09))
R_att = diag((8, 8, 8))
K_pi_alt = array((0.0013, 0.0747, 0.1068, -0.01))
maxIntegralInfluence = 0.001

### Drone Model and Controllers

In [ ]:
dp = buildDroneParamsAndMatrices()
d = Drone(dp)
controller = d.getController(Q_att, R_att, K_pi_alt, maxIntegralInfluence)

### Reference

In [ ]:
def get_reference(t):
    m = 0.5  # time scale factor

    qz = eul2quat((8*pi/180, 0, 0))
    qy = eul2quat((0, 8*pi/180, 0))
    qx = eul2quat((0, 0, 8*pi/180))
    q = eul2quat((0, 0, 0))

    if (t >= m * 1 and t < m * 3):
        q = quatmultiply(q, qz)
    if (t >= m * 5 and t < m * 7):
        q = quatmultiply(q, qy)
    if (t >= m * 9 and t < m * 11):
        q = quatmultiply(q, qx)

    if (t >= m * 13 and t < m * 15):
        q = quatmultiply(q, quatconjugate(qy))
    if (t >= m * 17 and t < m * 19):
        q = quatmultiply(q, quatconjugate(qx))
    if (t >= m * 21 and t < m * 23):
        q = quatmultiply(q, quatconjugate(qz))

    if (t >= m * 27 and t < m * 30):
        q = quatmultiply(q, qy)
    if (t >= m * 28 and t < m * 31):
        q = quatmultiply(q, qx)

    if (t >= m * 33 and t < m * 36):
        q = quatmultiply(q, quatconjugate(qx))
    if (t >= m * 34 and t < m * 37):
        q = quatmultiply(q, quatconjugate(qy))

    rr = DroneReference()
    rr.setOrientation(q)
    rr.setPosition((0, 0, 1.0 * (t >= m * 8)))
    return rr.asColVector()

ref_function = DroneReferenceFunction(get_reference)

### Initial Conditions and Integration Options

In [ ]:
x0 = d.getStableState()

odeopt = AdaptiveODEOptions()
odeopt.t_start = 0
odeopt.t_end = 16
odeopt.epsilon = 1e-6
odeopt.h_start = 1e-6
odeopt.h_min = 1e-8
odeopt.maxiter = int(1e6)

### Plotting

In [ ]:
result = d.simulate(controller, ref_function, x0, odeopt)
start_time = time.time()
fig, lines = DronePlotter.plot(result, vertical=False, w=1500, h=800)
end_time = time.time()
print('Initial plotting took {:.3f} ms'.format((end_time - start_time)*1000.0))

def update(**kwargs):
    start_time = time.time()
    dp = buildDroneParamsAndMatrices(**kwargs)
    assert(dp.uh <= 0.90)
    d = Drone(dp)
    K_pi_alt[3] = -kwargs['Ki_alt'] # Note the minus sign!
    controller = d.getController(Q_att, R_att, K_pi_alt, kwargs['maxInt'])
    result = d.simulate(controller, ref_function, x0, odeopt)
    end_time = time.time()
    print('Simulation took {:.3f} ms'.format((end_time - start_time)*1000.0))

    start_time = time.time()
    DronePlotter.update_plot(lines, result)
    fig.canvas.draw()
    end_time = time.time()
    print('Updating plot took {:.3f} ms'.format(
          (end_time - start_time)*1000.0))

In [ ]:
interact(update, 
         m=widgets.FloatSlider(min=1.0,max=3.0,step=0.1,value=1.8,continuous_update=False),
         tau_m=widgets.FloatSlider(min=0.010,max=0.050,step=0.001,value=0.035,readout_format='.3f',continuous_update=False),
         Ixx=widgets.FloatSlider(min=0.010,max=0.050,step=0.001,value=0.032,readout_format='.3f',continuous_update=False),
         Iyy=widgets.FloatSlider(min=0.010,max=0.050,step=0.001,value=0.032,readout_format='.3f',continuous_update=False),
         Izz=widgets.FloatSlider(min=0.020,max=0.090,step=0.001,value=0.058,readout_format='.3f',continuous_update=False),
         cp=widgets.FloatSlider(min=0.010,max=0.070,step=0.001,value=0.04,readout_format='.3f',continuous_update=False),
         L=widgets.FloatSlider(min=0.10,max=0.44,step=0.01,value=0.27,readout_format='.2f',continuous_update=False),
         maxInt=widgets.FloatLogSlider(min=-4,max=-2,step=0.1,value=maxIntegralInfluence,readout_format='.4f',continuous_update=False),
         Ki_alt=widgets.FloatLogSlider(min=-3,max=-1,step=0.1,value=0.01,readout_format='.4f',continuous_update=False));